## 객체 인식
OpenCV의 함수를 통한 객체 인식. 학습이 아닌 방법으로 객체를 인식하기 위해서는 특징점(Key Point)과 기술자(Descriptor)를 활용한다.

#### 특징점(Key Point)
* 특징점은 **영상에서 배경과 구분되면서 고유한 식별 지점**을 의미한다.
* 서로 다른 이미지에서도 하나 이상의 지점이 특별하게 구분할 수 있는 작은 부분을 가리킨다. 

#### 기술자(Descriptor)
* 서로 다른 이미지에서 특징점이 어떤 연관성을 가졌는지 구분하게 한다.
* 각 특징점이 가진 **지역적 특징 정보**를 갖고 있다.
* **서로 다른 특징점에서 차이를 구분해 특징점끼리 서로 매칭**되게 한다.
* 완전히 다른 이미지나 빠른 프레임에서도 객체를 추적할 수 있다.

객체 인식은 특징점과 기술자를 통해 이미지에서 객체를 조사하고, 객체와 관련된 특징점이 다수 존재한다면 해당 객체가 다른 이미지에서도 존재하는지 판단할 수 있다.

### ORB(Oriented FAST and rotated BRIEF)알고리즘
* FAST(Features from Accelerated Segment Test) 알고리즘, BRIEF(Binay Robust Independent Elementary Features) 알고리즘, 해리스 알고리즘을 결합한 알고리즘이다.
  * FAST(Features from Accelerated Segment Test) 알고리즘
    - 피처 검출기 알고리즘으로서 **픽셀 P와 픽셀 주변의 작은 원 위에 있는 픽셀의 집합을 비교하는 방식**이다.
    - 픽셀 P의 주변 픽셀에 임곗값을 적용해 어두운 픽셀, 밝은 픽셀, 유사한 픽셀로 분류해 원 위의 픽셀이 연속적으로 어둡거나 밝아야 하며 이 연속성이 절반 이상이 돼야 한다.
    - 이 조건을 만족하는 경우 해당 픽셀은 우수한 특징점으로 볼 수 있다는 개념이다.
  * BRIEF(Binay Robust Independent Elementary Features) 알고리즘
    - 특징점을 검출하는 알고리즘이 아닌 검출된 특징점에 대한 기술자를 생성하는 데 사용한다.
    - **특징점 주변 영역의 픽셀을 다른 픽셀과 비교해 어느 부분이 더 밝은지를 찾아 이진 형식으로 저장**한다.
    - 가우시안 커널을 사용해 이미지를 컨볼루션 처리하며, 피처 중심 주변의 가우스 분포를 통해 첫 번째 지점과 두 번째 지점을 계산해 모든 픽셀을 한 쌍으로 생성한다.
    - 즉, 두 개의 픽셀을 하나의 그룹으로 묶는 방식이다.
* ORB 알고리즘은 FAST 알고리즘을 사용해 특징점을 검출한다.
* FAST 알고리즘은 코너뿐만 아니라 가장자리에도 반응하는 문제점으로 인해 해리스 코너 검출 알고리즘을 적용해 최상위 특징점만 추출한다.
* 이 과정에서 이미지 피라미드를 구성해 스케일 공간 검색을 수행한다.
* 이후 스케일 크기에 따라 피처 주변 박스 안의 강도 분포에 대해 X축과 Y축을 기준으로 1차 모멘트를 계산한다.
* 1차 모멘트는 그래디언트 방향을 제공하므로 피처의 방향을 지정할 수 있다.
* 방향이 지정되면 해당 방향에 대해 피처 벡터를 계산할 수 있다.
* 피처는 회전 불변성을 갖고 있으며 방향 정보를 포함하고 있다.

#### Python OpenCV의 ORB(Oriented FAST and rotated BRIEF)클래스
```python
orb = cv2.ORB_create(
  nfeatures = 500,                      # 최대 피처 수, ORB 객체가 한번에 검출하고자 하는 특징점의 개수
  scaleFactor = 1.2f,                   # 스케일 계수, 이미지 피라미드 설정
  nlevels = 8,                          # 피라미드 레벨, 이미지 피라미드의 레벨 수
  edgeThreshold = 31,                   # 엣지 임계값, 이미지 테두리에서 발생하는 특징점을 무시하기 위한 경계의 크기
  firstLevel = 0,                       # 시작 피라미드 레벨, 원본 이미지를 넣을 피라미드의 레벨
  WTA_K = 2,                            # 비교점, BRIEF 기술자가 구성하는 비교 비트
  scoreType = cv2.ORB_HARRIS_SCORE,     # 점수 타입, 피처의 순위를 매기는 데 사용
  patchSize = 31,                       # 패치 크기, 방향성을 갖는 BRIEF 기술자가 사용하는 개별 피처의 패치 크기
  fastThreshold = 20                    # FAST 임계값, FAST 검출기에서 사용되는 임계값
)
```

#### Python OpenCV의 특징점 및 기술자 계산 메서드
```python
keypoints, descriptors = orb.detectAndCompute(
  image,                           # 입력 이미지
  mask,                            # 마스크 이미지
  descriptors = None,              # 기술자
  useProvidedKeypoints = False     # 특징점 사용, 참일 경우 특징점을 감지하는 대신 특징점을 입력으로 사용
)
```

#### Python OpenCV의 전수 조사 매칭 클래스
특징점과 기술자 검출이 완료되면 전수 조사 매칭을 활용해 객체를 인식하거나 추적할 수 있다.
```python
bf = cv2.BFMatcher(
  normType = cv2.NORM_L2,    # 거리 측정법, 질의 기술자와 훈련 기술자를 비교할 때 사용되는 거리 계산 측정법을 지정한다.
  crossCheck = False         # 교차 검사, 올바르지 않은 매칭을 제거하는데 효율적이지만 연산 시간이 증가한다.
)
```

#### Python OpenCV의 매치 함수
```python
DMatch = cv2.DescriptorMatcher.match(
  queryDescriptors,    # 질의 기술자
  trainDescriptors,    # 와 훈련 기술자를 사용해 최적의 매칭을 찾는다
  mask                 # 기술자 공간에서 작동하는 마스크의 행은 질의 기술자의 행과 대응하며 열은 내부 사전 이미지와 대응한다.
)
```

#### 예제 10.17 객체인식

In [ ]:
import cv2

img = cv2.imread('image.jpg')                                        # 검출하려는 객체가 포함된 이미지
img_object = cv2.imread('object.jpg', cv2.IMREAD_GRAYSCALE)          # 인식을 위한 객체 이미지
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)                         # 연산을 위해 단일 채널 이미지로 변형

orb = cv2.ORB_create(nfeatures=40000)                                # ORB 클래스 생성, 최대 피처 수 설정 : 피처 수가 많으면 속도는 느리지만 정확도 향상
kp1, des1 = orb.detectAndCompute(gray,None)                          # 특징점 및 기술자 계산
kp2, des2 = orb.detectAndCompute(img_object,None)                    # 특징점 및 기술자 계산

bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)                # 전수 조사 매칭, 해밍 거리법과 교차 검사를 진행해 기술자를 비교한다.
matches = bf.match(des1, des2)                                       
matches = sorted(matches, key=lambda x: x.distance)                  # 유효 거리가 짧을수록 품질이 우수한 매칭, 유효 거리가 짧은 순으로 정렬

count = 100
for i in matches[:count]:                                            # 우수한 매칭 지점 100개를 객체가 포함된 이미지 위에 표시한다
  idx = i.queryIdx
  x1, y1 = kp1[idx].pt
  cv2.circle(img, (int(x1), int(y1)), 3, (0,0,255), 3)               # 매칭이 우수한 지점 원그리기 함수로 표시

cv2.imshow('img', img)
cv2.waitKey(0)


#### Python OpenCV의 특징점 매칭 그리기 함수
특징점에 대한 매칭을 시각적으로 이미지 위에 표시하는 함수
```python
outImg = cv2.drawMatches(
  img1,                                     # 질의 이미지
  keypoints1,                               # 질의 이미지 특징점 
  img2,                                     # 훈련 이미지
  keypoints2,                               # 훈련 이미지 특징점
  matches1to2,                              # DMatch 객체
  outImg,                                   # 출력 이미지
  matchColor = None,                        # 매칭 색상
  singlePointColor = None,                  # 비매칭 색상
  matchesMask = None                        # 매치 마스크
  flags = cv2.DRAW_MATCHES_FLAGS_DEFAULT    # 플래그, 특징점과 매칭 정보에 대한 시각화 설정
)
```

#### 10.18 특징점 매칭 그리기 함수의 사용법

In [ ]:
flag = (cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS | cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)  # 매칭되지 않은 특징점을 시각화하지 않고, 매칭된 특징점은 크기와 방향 정보로 시각화하도록 설정
matching_result = cv2.drawMatches(img, kp1, img_object, kp2, matches[:count], None, flags=flag)      # DMatch 객체는 앞선 반복문에서 표시한대로 100개 할당

cv2.imshow("Matching result", matching_result)
cv2.waitKey(0)

* 출력 이미지에서 좌측 이미지는 객체가 포함된 질의 이미지이며, 우측 이미지는 검출할 객체인 훈련 이미지다.
* 출력 결과에서 확인할 수 있듯이 예제 10.17보다 매칭 결과를 더 직관적으로 확인할 수 있다.
* 특징점을 직접 조작하거나 응용하는 경우에는 예제 10.17과 같은 방식을 권장하며, 시각적으로 확인하는 경우에는 특징점 매칭 그리기 함수를 사용하는 방법을 권장한다.